In [ ]:
# | default_exp compute_tleaf

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *

In [12]:
# | export
import numpy as np
from typing import Dict
import collections

In [59]:
# | export

def compute_tleaf(t_air:float, # Air temperature (degC)
                  par:float, # Unknown parameter definition
                  potential_par:float, # Unknown parameter definition. Calculated using `potential_par` function?
                  wind_speed:float, # Unknown parameter definition (m/s)
                  relative_humidity:int, # Air relative_humidity(%)
                  gs:float, # Stomatal conductance
                  g_cuti:float, # leaf conductance
                  e_inst:float, # Unknown parameter definition
                  psi_leaf:float, # Unknown parameter definition
                  leaf_size:float = 50, # Characteristic dimension from vegetation params in mm i.e. 1 (pine needle) to 3000 (banana leaf)
                  leaf_angle:int = 45, # Leaf angle (depuis le plan horizontal : 0-90 deg)
                  turn_off_eb:bool = False, # Unknown parameter definition. Tleaf Energy balance?
                  transpiration_model: str = ["jarvis", "granier"], # Transpiration model type
                  ) -> Dict: # Dictionary with parameters

        "Compute leaf temperature and Vapour Pressure deficit"

        # Assert parameters -----------------------------------------------------
        assert  0 <= relative_humidity <= 100, 'relative_humidity must be a value between 0 and 100'

        assert  -40 <= t_air <= 70, 'Unrealistic air temperature, value must be a value between -40 and 70'

        assert isinstance(turn_off_eb, bool),'turn_off_eb must be boolean (True or False)'

        assert  0 <= leaf_angle <= 90, 'leaf_angle must be a value between 0 and 90'

        assert transpiration_model in [
        "jarvis",
        "granier",
        ], f'{transpiration_model} not a valid option, choose  "jarvis" or "granier"'

        # Constants -------------------------------------------------------------

        # Force minimum wind speed to avoid excessive heating
        wind_speed = np.maximum(wind_speed, 0.1)

        # Calculate short-wave radiation (W/m2) # from µmol/m2/s to Watts/m2
        short_wave_radiation = par*0.5495

        # Absorptance to short_wave_radiation (%)
        abs_short_wave_radiation = 0.5

        # # Unknown meaning of g_flat
        g_flat = 0.00662

        # Coefficient in rbl equation m
        g_cyl  = 0.00403

        # # Unknown meaning of j_flat
        j_flat = 0.5

        # Coefficient in rbl equation  none
        j_cyl = 0.6

        # Emissivity none
        emiss_leaf = 0.97

        # Stefan-Boltzman constant   W m-2 K-4
        stefan_boltzman_const = 5.6704e-8

        # Density of dry air kg/m3
        dry_air_density = 1.292

        # Heat capacity of dry air  J kg-1 K-1
        heat_capacity_dry_air = 1010

        # Psychrometric constant kPa K-1
        psychro_constant  = 0.066

        # Coefficient in esat equation kPa
        e_sat_coeff_a  = 0.61121

        # Coefficient in esat equation none
        e_sat_coeff_b  = 17.502

        # Coefficient in esat equation °C
        e_sat_coeff_z  = 240.97

        # Original comment found in the source code
        # VARAIBLE CALCULEES
        # rst  #  stomatal resistance s m-1 (not needed)
        # esat #  saturation vapor pressure    kPa
        # ea   # water vapor pressure of the air    kPa
        # em_air # air emissivity
        # s   # slope of esat/T curve    kPa oC-1
        # SWRabs #   absorbed short-wave radiation    W m-2
        # LWRin  # incoming long-wave radiation    W m-2
        # LWRouti # isothermal outgoing long-wave radiation    W m-2
        # Rni # isothermal net radiation    W m-2
        # rr # radiative resistance    s m-1
        # rblr # boundary-layer + radiative resistance    s m-1
        # ym #  modified psychrometric constant    kPa K-1
        # rbl # leaf boundary-layer resistance    s m-1
        # Delta_T  # leaf-to-air temperature difference    degC
        # Tleaf, Tleaf_NonLinear # leaf temperature    degC


        # Create cloud_cover var
        if potential_par > 0:

            cloud_cover = par/potential_par

        else:
            cloud_cover = 0

        if cloud_cover > 1:
            cloud_cover = 1

        # ; #kPa  Unknown meaning of e_sat
        e_sat = e_sat_coeff_a * np.exp(e_sat_coeff_b * t_air / (t_air + e_sat_coeff_z))

        # Unknown meaning of ea
        ea = e_sat * (relative_humidity / 100)

        # Unknown meaning of s
        s = e_sat * e_sat_coeff_b * e_sat_coeff_z / ((t_air + e_sat_coeff_z)**2)

        #  Unknown meaning of em_air
        em_air = ((1 - 0.84 * cloud_cover) * 1.31 * ((10 * ea / (t_air + 273.15))**0.14285714) + 0.84 * cloud_cover)

        # Update VPD with esat and ea (why?)
        vpd_x = e_sat - ea

        # Bilan radiatif --------------------------------------------------------

        # Radiation absorbed by leaves
        swr_abs = abs_short_wave_radiation * np.cos(leaf_angle * 3.1416 / 180) * short_wave_radiation

        # Incoming long-wave radiation (W m-2) for clear and cloudy sky
        lwr_in = em_air * stefan_boltzman_const * (t_air + 273.15)**4

        # Outcoming long-wave radiation (W m-2) for clear and cloudy sky
        lwr_outi = emiss_leaf * stefan_boltzman_const * (t_air + 273.15)**4

        # isothermal net radiation
        rni = swr_abs + lwr_in - lwr_outi

        # Radiative resistance
        rad_res = dry_air_density * heat_capacity_dry_air / (4 * emiss_leaf * stefan_boltzman_const * (t_air + 273.15)**3)

        # Boundary layer resistance
        if leaf_size > 3:

            # Unknown meaning of rbl
            rbl = 1 / (1.5 * g_flat * ((wind_speed**j_flat) / ((leaf_size / 1000)**(1 - j_flat))))

        else:
            # a needle, formula for a cylinder. I am assuming that this comment
            # belongs to the following line

            # Unknown meaning of rbl
            rbl = 1 / (1.5 * g_cyl * ((wind_speed**j_cyl) / ((leaf_size / 1000)**(1 - j_cyl)))) # A flat leaf if > 3mm

        # leaf boundary layer conductance in mmol/s/m2
        g_bl = 1 / rbl * 1000 * 40

        # Unknown meaning of rblr
        rblr = 1 / (1 / rbl + 1 / rad_res)

        # Include the gs term into the energy balance
        if transpiration_model == "jarvis":

            if (gs + g_cuti) > 0:

                # Unknown meaning of rst
                rst = 1/(gs + g_cuti)*1000*40

            else:

                # Unknown meaning of rst
                rst = 9999.99

        if transpiration_model == "granier":

            # Unknown meaning of g
            g = e_inst/vpd_x*101.3

            if g > 0:

                # Unknown meaning of rbl
                rst = 1/(g)*1000*40

            else:

                # Unknown meaning of rbl
                rst = 9999.99

        # Unknown meaning of ym
        ym = psychro_constant * (rst/rblr)

        # Compute Tleaf with linear approximation -------------------------------
        delta_t = (ym * rni * rblr/(dry_air_density * heat_capacity_dry_air) - vpd_x)/(s + ym)

        t_leaf = t_air + delta_t

        # Create a copy of t_leaf. I don't understand why this is done
        #t_leaf_copy =  t_leaf

        # Saturation vapour water pressure at Tair in Pa from Buck's equation
        e_sat_air = 611.21 * np.exp((18.678 - t_air/234.5)*t_air/(257.14 + t_air))

        # Vapour water pressure at Tair and RHair
        e_air = e_sat_air * relative_humidity/100

        # Calculate VPD air
        vpd_air = (e_sat_air - e_air)/1000

        # Saturation vapour water pressure at Tair in Pa from Buck's equation
        e_sat_leaf = 611.21 * np.exp((18.678 - t_leaf/234.5) * t_leaf/(257.14 + t_leaf))

        # Unknown meaning of e
        e = e_sat_leaf * np.exp(psi_leaf*2.16947115/(t_leaf+273.15))

        # effect of leaf water potential on e
        # vpd between leaf and air in kPa
        vpd_leaf = np.maximum(0, (e - e_air)/1000)


        if turn_off_eb is False:
            vecres = collections.defaultdict(list,{"t_leaf":t_leaf,
                                                   "g_bl":g_bl,
                                                   "vpd_leaf":vpd_leaf,
                                                   "vpd_air":vpd_air,
                                                   "delta_t":delta_t
                                                   }
                                             )

        # If turn off energy balance Tleaf = Tair
        if turn_off_eb is True:
            vecres = np.array([t_air, g_bl, vpd_leaf, vpd_air])

            vecres = collections.defaultdict(list,{"t_air":t_air,
                                                   "g_bl":g_bl,
                                                   "vpd_leaf":vpd_leaf,
                                                   "vpd_air":vpd_air,
                                                   }
                                             )

        return vecres



## Compute Tleaf

In [65]:
compute_tleaf(t_air = 60,
              par = 100,
              potential_par = 1,
              wind_speed = 3,
              relative_humidity = 60,
              gs = 1,
              g_cuti = 2,
              e_inst = 3,
              psi_leaf = 4,
              leaf_size = 50,
              leaf_angle = 45,
              turn_off_eb = False,
              transpiration_model = "jarvis",
              )

defaultdict(list,
            {'t_leaf': 60.394042704329365,
             'g_bl': 3076.697970227172,
             'vpd_leaf': 8.880103411681098,
             'vpd_air': 7.978058190496631,
             'delta_t': 0.39404270432936683})